In [1]:
import torch 

In [2]:
pt_path = '/workspace/data/multi_cue_dataset/pt/ASD/20160120_ASD_lat__V1-0001.pt'
pt = torch.load(pt_path, map_location='cpu')

In [26]:
frames = pt['frames']
print(f"Number of frames: {len(frames)}")

Number of frames: 69


In [29]:
frames.dtype

torch.uint8

In [37]:
for k, v in pt.items():
    print(f"{k}: {v.shape if isinstance(v, torch.Tensor) else type(v)}")

video_dir: <class 'str'>
frames: torch.Size([69, 3, 512, 512])
label: <class 'int'>
total_frames: <class 'int'>
img_shape: <class 'tuple'>
bbox_none_index: <class 'list'>
optical_flow: torch.Size([69, 512, 512, 2])
bbox: torch.Size([69, 4])
mask: torch.Size([69, 1, 512, 512])
keypoint: torch.Size([69, 17, 2])
keypoint_score: torch.Size([69, 17])


In [34]:
pt["video_dir"]

PosixPath('/workspace/data/segmentation_dataset_512/fold0/val/ASD/20160120_ASD_lat__V1-0001.mp4')

In [13]:
pt["optical_flow"].dtype

torch.float32

In [17]:
bbox = pt["bbox"]
bbox.dtype

torch.float32

In [25]:
bbox.shape

torch.Size([69, 4])

In [19]:
bbox.half()
bbox.dtype

torch.float32

In [20]:
keypoint = pt["keypoint"]
keypoint_score = pt["keypoint_score"]
keypoint.dtype, keypoint_score.dtype

(torch.float32, torch.float32)

In [21]:
mask = pt["mask"]
mask.dtype

torch.float32

In [42]:
import torch
import gzip
from pathlib import Path

# 假设 pt 是你的推理结果字典
# pt = {...}

# 类型转换 + 压缩准备
for k, v in pt.items():
    print(f"处理 {k}: {type(v)}")
    
    # 不需处理的基本类型
    if isinstance(v, (str, int, tuple)):
        continue

    # Path 类型转为字符串
    elif isinstance(v, Path):
        pt[k] = str(v)

    # Tensor 类型处理
    elif isinstance(v, torch.Tensor):
        if k == "mask":
            pt[k] = v.to(torch.uint8)
        elif v.is_floating_point() and v.dtype == torch.float32:
            try:
                pt[k] = v.to(torch.float16)
            except Exception as e:
                print(f"警告: 无法将 {k} 转换为 float16: {e}")

# 打印处理后的数据结构
print("\n✅ 保存前的数据结构：")
for k, v in pt.items():
    if isinstance(v, torch.Tensor):
        print(f"{k}: dtype={v.dtype}, shape={tuple(v.shape)}")
    else:
        print(f"{k}: type={type(v).__name__}, value={v}")

# 保存为 gzip 压缩的 .pt 文件
with gzip.open("inference_result.pt.gz", "wb") as f:
    torch.save(pt, f)

print("\n✅ 保存完成: inference_result.pt.gz")


处理 video_dir: <class 'str'>
处理 frames: <class 'torch.Tensor'>
处理 label: <class 'int'>
处理 total_frames: <class 'int'>
处理 img_shape: <class 'tuple'>
处理 bbox_none_index: <class 'list'>
处理 optical_flow: <class 'torch.Tensor'>
处理 bbox: <class 'torch.Tensor'>
处理 mask: <class 'torch.Tensor'>
处理 keypoint: <class 'torch.Tensor'>
处理 keypoint_score: <class 'torch.Tensor'>

✅ 保存前的数据结构：
video_dir: type=str, value=/workspace/data/segmentation_dataset_512/fold0/val/ASD/20160120_ASD_lat__V1-0001.mp4
frames: dtype=torch.uint8, shape=(69, 3, 512, 512)
label: type=int, value=0
total_frames: type=int, value=69
img_shape: type=tuple, value=(512, 512)
bbox_none_index: type=list, value=[]
optical_flow: dtype=torch.float16, shape=(69, 512, 512, 2)
bbox: dtype=torch.float16, shape=(69, 4)
mask: dtype=torch.uint8, shape=(69, 1, 512, 512)
keypoint: dtype=torch.float16, shape=(69, 17, 2)
keypoint_score: dtype=torch.float16, shape=(69, 17)



✅ 保存完成: inference_result.pt.gz
